## 네이버 채권분석 보고서 크롤링
https://finance.naver.com/research/debenture_list.naver?&page=147
- 20221118
- 수집기간 20080401~20221115
- pdf href를 이용한 크롤링 후 text 추출하기
    -  href="https://ssl.pstatic.net/imgstock/upload/research/debenture/1668729329628.pdf"

- 4,400개의 보고서 중 4,019개가 다운로드되었고 3,993개가 정상적인 pdf
- 3,993개의 pdf파일 변환 시도 -> 3,984개 txt변환완료

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 현재 경로 확인
import os
path=os.getcwd()
print(path)

/content


In [6]:
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen
import re
import datetime

bond_base_url='https://finance.naver.com/research/debenture_list.naver?'

#page지정해서 채권 리포트 pdf 다운로드받기-> 코랩에서 파일저장이 잘 안될경우, 주피터로 로컬에 저장하기.
#def naver_bond_pdf_download(page):
    
for page in range(148): #20221118기준, 총 147page존재

    page_url =bond_base_url+ "&page=" + str(page)
    page_req = urlopen(page_url)
    page_bs = BeautifulSoup(page_req, 'html.parser')
    #print(len(page_bs))

    #생성날짜, 증권사, 제목 가져오기
    bond_list=page_bs.select("div.box_type_m > table.type_1>tr")[1:]
    

    for bond in bond_list:
        bond_info_list=[]
        #print(bond)
        for bond_info in bond.select('td'):

            bond_info_list.append(bond_info)


        #print( bond_info_list)
        if len(bond_info_list)>1:
            title=bond_info_list[0].text #제목
            #print(title)
            company=bond_info_list[1].text #증권사이름
            #print(company)
            date=bond_info_list[3].text #생성날짜
            date = datetime.datetime.strptime(date, "%y.%m.%d").date()
            #print(date)

            try:
                #pdf download link   
                bond_pdf_download_href=bond_info_list[2].a.attrs['href']
                #print(bond_pdf_download_href)
                urlretrieve(bond_pdf_download_href,'./content/drive/MyDrive/naver_bond_1/' + str(date) + "_" + company + "_"+title + ".pdf")
            except Exception as e:
                print('error!', e)



## pdf to text

In [ ]:
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 3.7 MB/s 
     |████████████████████████████████| 3.2 MB 22.1 MB/s 
     |████████████████████████████████| 5.6 MB 19.4 MB/s 
     |████████████████████████████████| 142 kB 61.1 MB/s 
     |████████████████████████████████| 4.1 MB 50.5 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [ ]:
from urllib.request import urlretrieve
from urllib.request import Request
from urllib.request import urlopen
from urllib.request import urlretrieve
import re
import glob
import pdfplumber


all_files = (glob.glob("/content/drive/MyDrive/naver_bond_1/*.pdf"))
bond_all_text=[]

#변환된 txt파일 저장할 폴더경로
folder_dir="/content/drive/MyDrive/naver_bond_1/"

#속도문제로 파일 목록들을 나누어 돌림
for file in all_files[2170:]:
    file_name=file.split('/')[5] #pdf파일이름
    new_file_name = file_name.replace('.pdf', '.txt')#txt파일이름

    all_text=''
    try:
        with pdfplumber.open(file) as pdf:
            for page in pdf.pages: #pdf파일별 전체 페이지 가져오기
                #print(page)
                string = page.extract_text().replace('\n', '') #줄바꿈제거
                all_text = all_text + '\n' + string #전체페이지 합친 pdf별 전체텍스트
            
            #변환된 txt파일 저장
            with open(folder_dir+"/text/"+new_file_name, 'w',encoding='utf-8') as f:
                f.write(all_text)
                #print(string)

            bond_all_text.append(all_text) #pdf별로 빈리스트에 append

    except Exception as e:
        print(e)

print(bond_all_text)
len(bond_all_text)

can't concat float to bytes


bytes must be in range(0, 256)


'PSKeyword' object is not iterable


'NoneType' object has no attribute 'decode'


can't concat float to bytes


bytes must be in range(0, 256)


unsupported operand type(s) for +: 'float' and 'bytes'


bytes must be in range(0, 256)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1815

In [ ]:
#텍스트 파일 목록 확인
all_files = (glob.glob("/content/drive/MyDrive/naver_bond_reports/*.pdf"))
for file in all_files:
    file_name=file.split('/')[5]
    new_file_name = file_name.replace('.pdf', '.txt')
    print(new_file_name)

2022-11-18_신한투자증권_신한 Bond.Zip (11월 18일).txt
2022-11-18_유안타증권_채권 Daily (11.18).txt
2022-10-31_유안타증권_채권 Daily (10.31).txt
2022-11-02_유안타증권_채권 Daily (11.02).txt
2022-10-26_하나증권_급한 불은 껐다.txt
2022-10-28_유안타증권_채권 Daily (10.28).txt
2022-10-31_신한투자증권_오래가지 못할 피봇(Pivot) 랠리.txt
2022-10-27_유안타증권_채권 Daily (10.27).txt
2022-10-26_유안타증권_채권 Daily (10.26).txt
2022-10-25_유안타증권_채권 Daily (10.25).txt
2022-10-27_하나증권_한은의 11월 선택지 점검.txt
2022-11-16_하나증권_진화를 넘어 진정으로.txt
2022-11-18_대신증권_기준금리 인하기대라면너무 과도하다.txt
2022-11-16_하나증권_장기전도 내수가 버텨야 가능하다.txt
2022-11-17_유안타증권_채권 Daily (11.17).txt
2022-11-16_하이투자증권_한전채와 CP.txt
2022-11-16_하나증권_연준 손바닥 위에 놓인 미국 크레딧.txt
2022-11-04_신한투자증권_유턴(U-turn) 신호 기다리기.txt
2022-10-31_한화투자증ᄀ

## 변환된 채권보고서 txt 불러오기
- lineterminator : 각 행의 끝을 표시하기 위해 사용되는 문자

In [ ]:
# 모든 txt 파일 불러와서 파일별로 리스트에 담기

import glob
import re

all_txt_files = (glob.glob("/content/drive/MyDrive/naver_bond_1/text/*.txt"))
folder_dir="/content/drive/MyDrive/naver_bond_1/text/"
all_string_list=[]
for txt_file in all_txt_files:
    with open(txt_file,'r', encoding='utf-8') as f:
        string=f.readlines() #txt파일내 한줄씩 불러오기-> 리스트로 불러오기에 join필요
        new_string=' '.join(string)
        new_string=re.sub('\n','',new_string)

        #print(new_string)

        all_string_list.append(new_string)


len(all_string_list)

3984

In [ ]:
all_string_list[0]

' Credit 손소현 02. 3779-0055 thecredit@ebestsec.co.krRA 전병하 02. 3779-8633 byunghajeon@ebestsec.co.kr2015-10-16(금) Comment\uf0a7 피치, 브라질 신용등급 BBB에서 BBB-로 한 단계 강등  \uf0a7 미 뉴욕연방은행장, 예상대로 경제회복시 금리인상 선호 피치는 브라질 국가신용등급을 BBB에서 BBB-로 한 단계 강등.  미국 뉴욕 연방준비은행의 윌리엄 더들리 은행장이 통화정책과 BBB-는 피치가 분류하는 투자등급의 맨 아래 단계로 신용등급  관련해 현재 예상하는 속도대로 미국 경제가 회복된다면 기준금리를 전망을 부정적으로 제시하였음. 피치는 향후 정부 재정 악화가  올리는 쪽을 선호한다는 입장을 보임. 그러나 올해 금리를 올릴 수 지속되고, 경기 침체가 장기화 되면 추가 강등할 수 있다고 경고.  있을지에 대해서는 나도 알았으며 한다며 대답을 회피하는 모습을 무디스가 평가한 브라질 국가신용등급도 투자등급의 맨 아래 단계인  보였음. 더들리 은행장은 지금부터 연말 사이에 많은 자료가 나올 Baa3, 신용등급 전망은 안정적. S&P는 이미 지난달 투기등급인  예정이고 어떤 결정이든 그 자료에 따른다면서 최근에는 경제가 BB+로 강등하고 신용등급 전망을 부정적으로 제시하였음. 이후  둔화됐다는 소식도 있지만 국내 경제의 성과는 매우 좋다고 언급. 브라질 정부는 긴축과 증세를 통한 169억달러 규모 재정 확충 대책을  더들리 은행장은 연준 통화정책의 두 축인 물가와 고용동향 사이에 발표하였으며 내년 재정수지 흑자 목표치를 GDP 대비 0.5%에서  상관관계가 있다는 의견을 밝히기도 하였음 0.7%로 끌어올리는데 초점을 맞춤. 그러나 노동계, 재계, 연방의회가 반대하고 있어 의회 통과가 불투명한 상황  국내 채권 국내 크레딧(bp) 종가(%) 1D 1W 1M (bp) 금리(%) 스프레드 1D 1W 1M국내금리 CD(91일) 1.570 -1.0 -2.0  -3.0  특수채A

- 보고서 파일내 텍스트만 사용할 경우

In [ ]:
# 채권보고서의 모든 내용들을 하나의 텍스트 파일로 저장
all_string='\n'.join(all_string_list)
folder_dir="/content/drive/MyDrive/naver_bond_1/all_text/"

with open(folder_dir+'bond_reports_all_docs.txt', 'w',encoding='utf-8') as f:
    f.write(all_string)

In [ ]:
#txt파일 내 문서들을 '\n'으로 구분하려면 txt파일을 열어서 첫째줄에 '\n'삽입 후 불러와야함
import pandas as pd 
pd.read_table("/content/drive/MyDrive/naver_bond_1/all_text/bond_reports_all_docs_.txt",sep='\n',lineterminator='\n')

,\n
0,Credit 손소현 02. 3779-0055 thecredit@ebestsec.co...
1,2015년 10월 14일 I Global Asset Research ...
2,Credit 손소현 02. 3779-0055 thecredit@ebestsec.c...
3,2015년 10월 15일 I Global Asset Research ...
4,2015년 10월 13일 I Global Asset Research ...
...,...
3979,[10월 금통위] Correlation이 가장 두렵다 [Bond Str...
3980,"(단위: 억원, %)발간일: 10.12 (수)10.11 (화) 1d 5d 1M 전..."
3981,Fixed Income 2022.11.11 ` PF ABCP 유동성 ...
3982,I 1 Summary투자등급(IG) 및 하이일드(HY) 회사채 시장 전망 내...


- date, title, company가 담긴 데이터프레임으로 사용할 경우

In [ ]:
# 리스트를 데이터프레임으로 변경
import pandas as pd
docs_df=pd.DataFrame(all_string_list,columns=['docs'])
docs_df

,docs
0,Credit 손소현 02. 3779-0055 thecredit@ebestsec.c...
1,2015년 10월 14일 I Global Asset Research ...
2,Credit 손소현 02. 3779-0055 thecredit@ebestsec.c...
3,2015년 10월 15일 I Global Asset Research ...
4,2015년 10월 13일 I Global Asset Research ...
...,...
3979,[10월 금통위] Correlation이 가장 두렵다 [Bond Str...
3980,"(단위: 억원, %)발간일: 10.12 (수)10.11 (화) 1d 5d 1M 전..."
3981,Fixed Income 2022.11.11 ` PF ABCP 유동성 ...
3982,I 1 Summary투자등급(IG) 및 하이일드(HY) 회사채 시장 전망 내...


In [ ]:
# 날짜, 증권사, 제목을 붙이기 위해 파일 이름 가져와서 딕셔너리로 붙이기
import pandas as pd
import glob
all_txt_files = (glob.glob("/content/drive/MyDrive/naver_bond_1/text/*.txt"))

file_header_list=[]
for file in all_txt_files:
    file_name=file.split('/')[6]
    file_name = file_name.replace('.txt', '')
    date=file_name.split('_')[0]
    company=file_name.split('_')[1]
    title=file_name.split('_')[2]
    file_header_list.append({'date':date,'cmp':company,'title':title})

    #print(file_name)

# 딕셔너리를 데이터프레임으로 변환
file_header_df=pd.DataFrame(file_header_list)
file_header_df

,date,cmp,title
0,2015-10-16,이베스트투자증권,"피치, 브라질 신용등급 BBB에서 BBB-로 한 ..."
1,2015-10-14,하나증권,"불안감이 높아질 때, 더 빛나는 선진통..."
2,2015-10-15,이베스트투자증권,"Fed 베이지북, 달러강세 타격, 경기 완만ᄒ..."
3,2015-10-15,하나증권,정책에 대한 기대가 높아질 때 해야할 일
4,2015-10-13,하나증권,"단기 베어스팁, 국고10년은 2.20%을 넘지 ..."
...,...,...,...
3979,2022-10-12,하이투자증권,Correlation이 가장 두렵다
3980,2022-10-12,유안타증권,채권 Daily (10.12)
3981,2022-11-11,유안타증권,PF ABCP 유동성 점검
3982,2022-11-08,신한투자증권,Crescendo


In [ ]:
bond_reports_df=pd.concat([file_header_df,docs_df],axis=1)
bond_reports_df['date']=pd.to_datetime(bond_reports_df['date'])
bond_reports_df=bond_reports_df.sort_values('date').reset_index(drop=True)
bond_reports_df

,date,cmp,title,docs
0,2008-04-01,대우증권,대우증권 월간 채권투자 (4월),2008_04 월간채권투자채 권 시 장 전 망 對應과 限界금 융 시 장 차 트 ...
1,2008-04-07,대우증권,돌아온 박스권,Fixed Income Weekly 2008. 4. 7. #918 Fixed I...
2,2008-04-07,대우증권,시간차 공격,Fixed Income Weekly 2008. 4. 7. #918 Fixed I...
3,2008-04-21,대우증권,항상 열려는 있는 가능성,항상 열려는 있는 가능성 2008. 4. 21 Fixed Income...
4,2008-04-28,대우증권,FOMC 이후 원자재 가격 상승세 둔화 여부에 ...,임박한 기회? 2008. 4. 28 Fixed Income Weekl...
...,...,...,...,...
3979,2022-11-14,한화투자증권,[Hanwha FI Weekly] 울다가 웃으면 뿔이 날 ...,2022년 11월 14일 Hanwha Fixed Income Hanwha F...
3980,2022-11-14,신한투자증권,회복될 이유가 있는 투자심리,ㅇㅇ ㅇㅇ 신한생각 I 채권 전략 2022년 11월 14일...
3981,2022-11-14,유안타증권,채권 Daily (11.14),"(단위: 억원, %)발간일: 11.14 (월)11.11 (금) 1d 5d 1M 전..."
3982,2022-11-15,메리츠증권,"[채권전망] Era of Un-, 안전자산의 가치",2022년 11월 10일(목) Strategy IdeaStrategy Idea ...


In [ ]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 20.2 MB/s 


In [ ]:
# 최종 데이터프레임 저장
bond_reports_df.to_excel('/content/drive/MyDrive/naver_bond_1/all_text/bond_reports_all_df.xlsx',encoding='cp949', index=True)